# 可迭代的对象、迭代器和生成器

## 14.1 Sentence类第1版：单词序列

In [7]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

In [8]:
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [9]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [10]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

## 14.3 Sentence类第2版：典型的迭代器

In [13]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0
    
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word

    def __iter__(self):
        return self

## 14.4 Sentence类第3版：生成器函数

In [1]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word
        return

In [2]:
def gen_123():
    yield 1
    yield 2
    yield 3

In [3]:
gen_123()

<generator object gen_123 at 0x0000017244C47D10>

In [4]:
for i in gen_123():
    print(i)

1
2
3


In [5]:
g = gen_123()

In [6]:
print(next(g))
print(next(g))
print(next(g))

1
2
3


In [7]:
print(next(g))

StopIteration: 

In [9]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

In [10]:
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


## 14.5 Sentence类第4版：惰性实现

In [11]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group()

## 14.6 Sentence类第5版：生成器表达式

In [12]:
def gen_AB():
    print('Start')
    yield 'A'
    print('Continue')
    yield 'B'
    print('End.')

In [13]:
res1 = [x*3 for x in gen_AB()]

Start
Continue
End.


In [14]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [16]:
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x00000172467F44A0>

In [17]:
for i in res2:
    print('-->', i)

Start
--> AAA
Continue
--> BBB
End.


In [18]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self) -> str:
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))

## 14.8 另一个示例：等差数列生成器

In [19]:
class ArithmeticProgression:

    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> "infinite" series

    def __iter__(self):
        result = type(self.begin + self.step)(self.begin)
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result
            index += 1
            result = self.begin + self.step * index

In [20]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [22]:
import itertools
gen = itertools.count(1, .5)

In [23]:
for i in gen:
    print(i)
    if i > 3:
        break

1
1.5
2.0
2.5
3.0
3.5


In [24]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

In [25]:
import itertools

def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is not None:
        ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
    return ap_gen